# Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1:
*   Alumno 2:
*   Alumno 3:






---
## **PARTE 1** - Instalación y requisitos previos

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive en el caso de estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.


---
### 1.1. Preparar enviroment (solo local)



> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. En Windows, puede ser necesario instalar las C++ Build Tools. Para ello, siga los siguientes pasos: https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30.
2. Instalar Anaconda
3. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.


```
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
conda install git
pip install jupyter
```


4. Abrir la notebook con *jupyter-notebook*.



```
jupyter-notebook
```


---
### 1.2. Localizar entorno de trabajo: Google colab o local

In [2]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto practico"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

---
### 1.3. Montar carpeta de datos local (solo Colab)

In [3]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

Archivos en el directorio: 
['.git', '.gitignore', 'alternativa_google_collab.txt', 'checkpoints', 'models', 'Proyecto_practico.ipynb', 'Proyecto_practico_colab.ipynb', 'README.md']


---
### 1.4. Instalar librerías necesarias

In [5]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.8
else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install Keras==2.2.4
  %pip install tensorflow-gpu==2.5.3
  %pip install torch==2.0.1
  %pip install agents==1.4.0

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/Kojoley/atari-py.git to c:\users\davidv\appdata\local\temp\pip-req-build-dc8ihm5t
  Resolved https://github.com/Kojoley/atari-py.git to commit 86a1e05c0a95e9e6233c3a413521fdb34ca8a089
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/Kojoley/atari-py.git 'C:\Users\DavidV\AppData\Local\Temp\pip-req-build-dc8ihm5t'


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


...

  Using cached numpy-1.19.5-cp38-cp38-win_amd64.whl.metadata (2.0 kB)
  Using cached absl_py-0.15.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl.metadata (872 bytes)
  Using cached tensorflow_estimator-2.5.0-py2.py3-none-any.whl.metadata (1.2 kB)
  Using cached grpcio-1.34.1-cp38-cp38-win_amd64.whl.metadata (4.0 kB)
INFO: pip is looking at multiple versions of tensorboard to determine which version is compatible with other requirements. This could take a while.
  Using cached tensorboard-2.14.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorboard-2.12.3-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorboard-2.12.2-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorboard-2.12.1-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorboard-2.12.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorboard-2.11.2-py3-none-any.whl.metadata (1.9 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl.metadata (2.7 

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
matplotlib 3.7.5 requires numpy<2,>=1.20, but you have numpy 1.19.5 which is incompatible.
tensorflow-intel 2.13.0 requires absl-py>=1.0.0, but you have absl-py 0.15.0 which is incompatible.
tensorflow-intel 2.13.0 requires flatbuffers>=23.1.21, but you have flatbuffers 1.12 which is incompatible.
tensorflow-intel 2.13.0 requires keras<2.14,>=2.13.1, but you have keras 2.2.4 which is incompatible.
tensorflow-intel 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 1.19.5 which is incompatible.
tensorflow-intel 2.13.0 requires tensorboard<2.14,>=2.13, but you have tensorboard 2.11.2 which is incompatible.
tensorflow-intel 2.13.0 requires tensorflow-estimator<2.14,>=2.13.0, but you have tensorflow-estimator 2.5.0 which is incompatible.


Note: you may need to restart the kernel to use updated packages.


  Using cached absl_py-2.3.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached numpy-1.24.3-cp38-cp38-win_amd64.whl.metadata (5.6 kB)
  Using cached tensorboard-2.13.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorflow_estimator-2.13.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached keras-2.13.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached grpcio-1.70.0-cp38-cp38-win_amd64.whl.metadata (4.0 kB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
Using cached numpy-1.24.3-cp38-cp38-win_amd64.whl (14.9 MB)
Using cached absl_py-2.3.0-py3-none-any.whl (135 kB)
Using cached flatbuffers-25.2.10-py2.py3-none-any.whl (30 kB)
Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)
Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
Using cached grpcio-1.70.0-cp38-cp38-win_amd64.whl (4.3 MB

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-gpu 2.5.3 requires absl-py~=0.10, but you have absl-py 2.3.0 which is incompatible.
tensorflow-gpu 2.5.3 requires flatbuffers~=1.12.0, but you have flatbuffers 25.2.10 which is incompatible.
tensorflow-gpu 2.5.3 requires grpcio~=1.34.0, but you have grpcio 1.70.0 which is incompatible.
tensorflow-gpu 2.5.3 requires numpy~=1.19.2, but you have numpy 1.24.3 which is incompatible.
tensorflow-gpu 2.5.3 requires tensorflow-estimator<2.6.0,>=2.5.0, but you have tensorflow-estimator 2.13.0 which is incompatible.


---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [4]:
from __future__ import division

from PIL import Image, ImageEnhance
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

print(gym.__version__)

0.17.3


#### Configuración base

In [5]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [6]:
from tensorflow.python.client import device_lib
from tensorflow import config

list_gpu = config.list_physical_devices('GPU')
print(list_gpu)
if list_gpu:
    try:
        for gpu in list_gpu:
            config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [7]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        #processed_observation = np.array(img)

        ### Mejora del contraste de la imagen
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(1.5)  # 1.0 = sin cambio; >1.0 = más contraste
        processed_observation = np.array(img)
        #processed_observation = (processed_observation * 255).astype('uint8')  # volver a uint8

        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

1. Implementación de la red neuronal

In [8]:
model = Sequential()
model.add(Permute((2,3,1), input_shape = (4,84,84)))
model.add(Convolution2D(32, (8,8), strides = (4,4), activation = 'relu'))
model.add(Convolution2D(64,(4,4), strides = (2,2), activation = 'relu'))
model.add(Convolution2D(64,(3,3), strides = (1,1), activation = 'relu'))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(nb_actions, activation='linear'))

2. Implementación de la solución DQN

In [9]:

processor = AtariProcessor()
memory = SequentialMemory(limit=500000, window_length = 4)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=0.3,
                              value_min=0.01, value_test=0.01, nb_steps=250000)


DQN = DQNAgent(model=model,
               nb_actions = nb_actions,
               memory = memory,
               nb_steps_warmup = 50000,
               target_model_update = 1000,
               policy = policy,
               gamma = 0.99,
               train_interval = 4,
               batch_size=64,
               enable_double_dqn=True,
               processor = processor)

DQN.compile(Adam(learning_rate = 0.00001),metrics=['mae'])  


In [10]:
checkpoint_dir = 'models/v4/checkpoints/'

os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_callback = ModelIntervalCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'dqn_{step}.h5f'),
    interval=25000,
)

In [58]:
# Testing part to calculate the mean reward
weights_filename = 'models/v3/dqn_v3_weights.h5f'.format(env_name)
DQN.load_weights(weights_filename)

In [59]:
DQN.fit(env, nb_steps=1000000, visualize=False, verbose=2, callbacks=[checkpoint_callback])
DQN.save_weights('models/v3/dqn_v3_weights.h5f', overwrite=True)

Training for 1000000 steps ...


c:\Users\DavidV\anaconda3\envs\miar_rl\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


    593/1000000: episode: 1, duration: 3.155s, episode steps: 593, steps per second: 188, episode reward: 10.000, mean reward:  0.017 [ 0.000,  1.000], mean action: 1.874 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   1438/1000000: episode: 2, duration: 4.235s, episode steps: 845, steps per second: 200, episode reward: 17.000, mean reward:  0.020 [ 0.000,  1.000], mean action: 1.978 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   2291/1000000: episode: 3, duration: 4.251s, episode steps: 853, steps per second: 201, episode reward: 16.000, mean reward:  0.019 [ 0.000,  1.000], mean action: 1.967 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   2672/1000000: episode: 4, duration: 1.914s, episode steps: 381, steps per second: 199, episode reward:  6.000, mean reward:  0.016 [ 0.000,  1.000], mean action: 1.900 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   3673/1000000: episode: 5, duration: 4.926s, episode steps: 1001, step

c:\Users\DavidV\anaconda3\envs\miar_rl\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


  50268/1000000: episode: 66, duration: 15.945s, episode steps: 718, steps per second:  45, episode reward: 16.000, mean reward:  0.022 [ 0.000,  1.000], mean action: 2.006 [0.000, 5.000],  loss: 0.019849, mae: 2.551162, mean_q: 3.072429, mean_eps: 0.241845
  50959/1000000: episode: 67, duration: 31.464s, episode steps: 691, steps per second:  22, episode reward: 16.000, mean reward:  0.023 [ 0.000,  1.000], mean action: 2.080 [0.000, 5.000],  loss: 0.017493, mae: 2.544481, mean_q: 3.064133, mean_eps: 0.241290
  51925/1000000: episode: 68, duration: 44.127s, episode steps: 966, steps per second:  22, episode reward: 20.000, mean reward:  0.021 [ 0.000,  1.000], mean action: 1.925 [0.000, 5.000],  loss: 0.017572, mae: 2.545332, mean_q: 3.064372, mean_eps: 0.240327
  53193/1000000: episode: 69, duration: 55.972s, episode steps: 1268, steps per second:  23, episode reward: 26.000, mean reward:  0.021 [ 0.000,  1.000], mean action: 2.106 [0.000, 5.000],  loss: 0.017048, mae: 2.537344, mean

In [60]:
DQN.save_weights('models/v4/dqn_v4_weights.h5f', overwrite=True)

In [ ]:
# Testing part to calculate the mean reward
weights_filename = 'models/v4/dqn_v4_weights.h5f'.format(env_name)
DQN.load_weights(weights_filename)
hist = DQN.test(env, nb_episodes=10, visualize=False)
rewards = hist.history.get('episode_reward',[])

print("Mean reward:", np.mean(rewards))

Testing for 10 episodes ...
Episode 1: reward: 27.000, steps: 900
Episode 2: reward: 24.000, steps: 1019
Episode 3: reward: 18.000, steps: 801
Episode 4: reward: 25.000, steps: 998
Episode 5: reward: 24.000, steps: 800
Episode 6: reward: 24.000, steps: 814
Episode 7: reward: 26.000, steps: 992
Episode 8: reward: 17.000, steps: 613
Episode 9: reward: 25.000, steps: 1146
Episode 10: reward: 22.000, steps: 831
Mean reward: 23.2


3. Justificación de los parámetros seleccionados y de los resultados obtenidos

---